```{index} single: Pyomo; variables 
```
```{index} single: Pyomo; expressions 
```
```{index} single: Pyomo; sets 
```
```{index} single: Pyomo; decorators
```
```{index} single: solver; cbc
```

# A Data-Driven AMPL Model

In this notebook, we'll revisit the production planning example. However, this time we'll demonstrate how Python's data structures, combined with Pyomo's capabilities, can create an optimization model scales with problem data. This enables the model to adjust to new products, varying prices, or changing demand. We refer to this as "data-driven" modeling.

The additional Pyomo components used in this notebook are:

* [Sets](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Sets.html)
* [Parameters](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Parameters.html)

These components enable the use indexed variables and constraints. The combination of sets and indices are essential to building scalable and maintainable models for more complex applications. 

We will begin this analysis by examining the problem data sets to identify the underlying problem structure.

In [66]:
# install dependencies and select solver
!pip install -q amplpy matplotlib pandas

SOLVER = "highs"

## Data representations

We begin by revisiting the data sets and mathematical model developed for the basic production planning problem presented in the previous notebook. The original data sets were given as:

| Product | Material <br> required | Labor A <br> required | Labor B <br> required | Market <br> Demand | Price |
| :-: | :-: | :-: | :-: | :-: | :-: | 
| U | 10 g | 1 hr | 2 hr | $\leq$ 40 units | 270€ |
| V |  9 g | 1 hr | 1 hr | unlimited | 210€ |

| Resource | Amount <br> Available | Cost | 
| :-: | :-: | :-: |
| Material | ? | 10€ / g |
| Labor A | 80 hours | 50€ / hour |
| Labor B | 100 hours | 40€ / hour | 

Two distinct sets of objects are evident from these tables. The first is the set of products comprised of $U$ and $V$. The second is the set of resources used to produce those products, which we abbreviate as $M$, $A$, and $B$. 

Having identified these sets, the data for this application be factored into three simple tables. The first two tables list attributes of the products and resources, the third table summarizes the process used to create the products from the resources:

**Table: Products**

| Product | Demand | Price |
| :-: | :-: | :-: | 
| U |  $\leq$ 40 units | 270€ |
| V |  unlimited | 210€ |

**Table: Resources**

| Resource | Available | Cost | 
| :-: | :-: | :-: |
| Material | ? | 10€ / g |
| Labor A | 80 hours | 50€ / hour |
| Labor B | 100 hours | 40€ / hour | 

**Table: Processes**

| Product | Material | Labor A | Labor B 
| :-: | :-: | :-: | :-: |
| U | 10 g | 1 hr | 2 hr |
| V |  9 g | 1 hr | 1 hr |


Python has many built-in data types and libraries that are useful for handling tabular data, and there are several options that would be appropriate for the task at hand. Nested dictionaries can be a good choice for smaller problems that have only a few columns. In the following examples, we will show how nested dictionaries can be used to represent the three tables that were described above.

The first table of these describes the products. The product names will serve as keys for outermost dictionary, and attribute names as keys for the inner dictionaries. The attribute values will be interpreted as floating point numbers. `None` is used when a value is not present.

In [67]:
products = {
    "U": {"price": 270, "demand": 40},
    "V": {"price": 210, "demand": None},
}

# print data
for product, attributes in products.items():
    for attribute, value in attributes.items():
        print(f"{product} {attribute:10s} {value}")

U price      270
U demand     40
V price      210
V demand     None


The second table is the nested dictionary listing attributes and values for resources consumed. 

In [68]:
resources = {
    "M": {"price": 10, "available": None},
    "labor A": {"price": 50, "available": 100},
    "labor B": {"price": 40, "available":  80},
}

for resource, attributes in resources.items():
    for attribute, value in attributes.items():
        print(f"{resource:8s} {attribute:10s} {value}")

M        price      10
M        available  None
labor A  price      50
labor A  available  100
labor B  price      40
labor B  available  80


The third table data shows the amount of each resource needed to produce one unit of each product. The rows are labeled by product, and the columns labeled by by resource. 

In [69]:
processes = {
    "U": {"M": 10, "labor A": 2, "labor B": 1},
    "V": {"M":  9, "labor A": 1, "labor B": 1},
}

for product, process in processes.items():
    for resource, value in process.items():
        print(f"{product:4s} {resource:10s} {value}")

U    M          10
U    labor A    2
U    labor B    1
V    M          9
V    labor A    1
V    labor B    1


## Mathematical model

By rearranging the problem data into straightforward tables, the structure of the production planning problem becomes evident. We can identify a set of products, a set of resources, and a collection of parameters that specify the processes for transforming resources into products. Compared to the previous notebook, these abstractions allow us to create mathematical models that can adapt and scale with the supplied data.

Let $\cal{P}$ and $\cal{R}$ be the set of products and resources, respectively, and let $p$ and $r$ be representative elements of those sets. We use indexed decision variables $x_r$ and $y_p$ to denote the amount of resource $r$ that is consumed in production, and $y_p$ to denote the amount of product $p$ produced. 

The problem data provides attributes that constrain the decision variables. For example, the decision variables all have lower bounds of zero, and some have upper bounds. We represent these as

$$
\begin{aligned}
    0 \leq x_r \leq b_r & & \forall r\in\cal{R} \\
    0 \leq y_p \leq b_p & & \forall p\in\cal{P} \\
\end{aligned}
$$

where the upper bounds, $b_r$ and $b_q$, come from the tables of attributes. For cases where the upper bounds don't apply, we can either insert bounds larger than would ever be encountered, or, when we translate this model to Pyomo, designate a special value that causes the bound to be ignored.

The objective is given as before,

$$
\begin{aligned}
    \text{profit} & = \text{revenue} - \text{cost} \\
\end{aligned}
$$

but now the expressions for revenue and cost are

$$
\begin{aligned}
    \text{revenue} & = \sum_{p\in\cal{P}} c_p y_p  \\
    \text{cost} & = \sum_{r\in\cal{R}} c_r x_r \\
\end{aligned}
$$

where $c_r$ and $c_p$ are parameters specifying the price for resources and products. The bounds on available resources can be written as

$$
\begin{aligned}
    \sum_{p\in\cal{P}} a_{r, p} y_p & \leq x_r & \forall r\in\cal{R}
\end{aligned}
$$

Putting these pieces together, we have the following model for the production planning problem.

$$
\begin{align}
\max \quad & \text{profit} = \sum_{p\in\cal{P}} c_p y_p - \sum_{r\in\cal{R}} c_r x_r \\
\text{such that} \quad & \sum_{p\in\cal{P}} a_{r, p} y_p  \leq x_r & \forall r\in\cal{R} \nonumber \\
 &   0 \leq x_r \leq b_r & \forall r\in\cal{R} \nonumber  \\
 &   0 \leq y_p \leq b_p & \forall p\in\cal{P} \nonumber  \\
\end{align}
$$

Compared to the previous notebook, when formulated this way, the model can be applied to any problem with the same structure, regardless of the number of products or resources. This flexibility is possible due to the use of sets to describe the products and resources for a particular problem, indices to refer to elements of those sets, and data tables that hold the relevant parameter values.

Generalizing mathematical models in this fashion is a common feature of many data science applications. Next we will see how this type of generalization is facilitated in Pyomo.

## The production model in AMPL

As before, we begin the construction of a Pyomo model by creating a `ConcreteModel`.

In [ ]:
from amplpy import AMPL, tools

ampl = tools.ampl_notebook(
    modules=["highs"], # modules to install
    license_uuid="default", # license to use
    g=globals()) # instantiate AMPL object and register magics

Next we use Pyomo `Set()` to construct sets corresponding to the products and resources. Each set is initialized with the dictionary keys for the relevant attribute tables. These will later become indices for parameters, decision variables, and constraints.

In [71]:
%%ampl_eval

set PRODUCTS;
set RESOURCES;

#model.PRODUCTS = pyo.Set(initialize=products.keys())
#model.RESOURCES = pyo.Set(initialize=resources.keys())

The next step is to introduce parameters that will be used in the constraints and objective functions. These are indexed by products, or resources, or both. The parameter values are assigned to model name. We use Pyomo decorators to declare these parameters, where the function between decorated becomes the name of the parameter, and the function returns the parameter value from the problem data sets. This forms the interface between problem data and the Pyomo model.

This step of declaring Pyomo `Param` objects is often omitted in Pyomo applications. In doing so, the modeler is making a choice to embed the external data representation directly into objectives and constraints of the problem. This can be effective, it does keep the code shorter, and may remove some computational overhead. However, it also blurs the boundary between the data representation and the model statements. Any change in data representation may require editing every place where that data is used in the model. By defining model parameters with `Param()`, the interface to the data representation is located in one clearly defined portion of a larger model, thereby significantly improving the long-term maintainabiity of models. This concern may be overkill in small models like we have here, but is a key consideration when constructing more complex applications.

Note: The domain for the bounds is set to `Any` because some of them will take value `None`. Pyomo will omit a lower or upper bounds that has a value of `None`, so this is a way to keep the logic simple.

In [72]:
%%ampl_eval

param demand{PRODUCTS};
param available{RESOURCES};

param cp{PRODUCTS};
param cr{RESOURCES};

param a{RESOURCES, PRODUCTS};

## parameter for bounds
#@model.Param(model.PRODUCTS, domain=pyo.Any)
#def demand(model, product):
#    return products[product]['demand']
#
#@model.Param(model.RESOURCES, domain=pyo.Any)
#def available(model, resource):
#    return resources[resource]['available']
#
## parameter with price coefficients
#@model.Param(model.PRODUCTS)
#def cp(model, product):
#    return products[product]['price']
#
#@model.Param(model.RESOURCES)
#def cr(model, resource):
#    return resources[resource]['price']
#
## process parameters: a[r,p]
#@model.Param(model.RESOURCES, model.PRODUCTS)
#def a(model, resource, product):
#    return processes[product][resource]

The decision variables, $x$ and $y$, are indexed by the set of resources and products, respectively. The indexing is specified by passing the relevant sets as the first argumentns to Pyomo `Var()`. In addition to indexing, it always good practice to specify any known and fixed bounds on the variables. This is done by specifying a function (in Pyomo parlance sometimes called a rule) that returns the bound for a given index. Here we use a Python lambda function with two arguments, model and an index referring to a member of a set, to return a tuple with the lower and upper bound.

In [73]:
%%ampl_eval

var x{r in RESOURCES} >= 0, <= available[r];
var y{p in PRODUCTS} >= 0, <= demand[p];

#model.x = pyo.Var(model.RESOURCES, bounds=lambda model, resource: (0,  model.available[resource]))
#model.y = pyo.Var(model.PRODUCTS, bounds=lambda model, product: (0, model.demand[product]))

The objective is expressed with Pyomo `quicksum` which accepts a Python generator to successive terms in the sum. Here we use the parameters $c_p$ and $c_r$ that appear in the mathematical version of the model, and which were declared above in the Pyomo version of the model.

In [74]:
%%ampl_eval

var revenue = sum{p in PRODUCTS} cp[p] * y[p];
var cost = sum{r in RESOURCES} cr[r] * x[r];

# create objective
maximize profit: revenue - cost;

#model.revenue = pyo.quicksum(model.cp[product] * model.y[product] for product in model.PRODUCTS)
#model.cost = pyo.quicksum(model.cr[resource] * model.x[resource] for resource in model.RESOURCES)
#
## create objective
#@model.Objective(sense=pyo.maximize)
#def profit(model):
#    return model.revenue - model.cost

The Pyomo `Constraint` decorator accepts one or more sets as arguments. Then, for every member of every set, the decorated function creates an associated constraint. Creating indexed constraints indexed in this manner are an essential building block for more complex models.

In [75]:
%%ampl_eval

# create indexed constraint
s.t. materials_used {r in RESOURCES}:
    sum{p in PRODUCTS} a[r, p] * y[p] <= x[r];

## create indexed constraint
#@model.Constraint(model.RESOURCES)
#def materials_used(model, resource):
#    return pyo.quicksum(model.a[resource, product] * model.y[product] for product in model.PRODUCTS) <= model.x[resource]

The final step is to solve the model and report the solution. Here we create a simple report using `pyo.value()` to access values of the decision variables, and using the model sets to construct iterators to report the value of indexed variables.

In [76]:
# load set data
ampl.set["PRODUCTS"] = list(products.keys())
ampl.set["RESOURCES"] = list(resources.keys())

# extract data from nested dictionaries
cp = {k : v["price"] for k, v in products.items()}
demand = {k : v["demand"] for k, v in products.items()}

cr = {k : v["price"] for k, v in resources.items()}
available = {k : v["available"] for k, v in resources.items()}

a = {(k2, k) : v2 for k, v in processes.items() for k2, v2 in v.items()}

# None is interpreted as an infinite value
INF = float("inf")

for k, v in demand.items():
    if not v:
        demand[k] = INF

for k, v in available.items():
    if not v:
        available[k] = INF
        
# load data for parameters
ampl.param["cp"] = cp
ampl.param["demand"] = demand
ampl.param["cr"] = cr
ampl.param["available"] = available
ampl.param["a"] = a

# set solver and solve
ampl.option["solver"] = SOLVER
ampl.solve()

# create a solution report
print(f"Profit = {ampl.obj['profit'].value()}")

print("\nProduction Report")
y = ampl.var["y"].get_values()

for product in y:
    print(f" {product[0]}  produced =  {product[1]}")
    
print("\nResource Report")
x = ampl.var["x"].get_values()

for resource in x:
    print(f" {resource[0]} consumed = {resource[1]}")

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 2400
3 simplex iterations
0 barrier iterations
Profit = 2400.0

Production Report
 U  produced =  20.0
 V  produced =  60.0

Resource Report
 M consumed = 740.0
 labor A consumed = 100.0
 labor B consumed = 80.0


## For Python experts: Creating subclasses of `AMPL`

Some readers of these notebooks may be more experienced Python developers who wish to apply Pyomo in more specialized, data driven applications. The following cell shows how the Pyomo `ConcreteModel()` class can be extended by subclassing to create specialized model classes. Here we create a subclass called `ProductionModel` that accepts a particular representation of the problem data to produce a production model object. The production model object inherits all of the methods associated with any `ConcreteModel`, such as `.display()`, `.solve()`, and `.pprint()`, but can be extended with additional methods.

In [77]:
%%writefile production_planning.mod

# sets
set PRODUCTS;
set RESOURCES;

# parameters
param demand{PRODUCTS};
param cp{PRODUCTS};
param available{RESOURCES};
param cr{RESOURCES};
param a{RESOURCES, PRODUCTS};

# variables
var x{r in RESOURCES} >= 0, <= available[r];
var y{p in PRODUCTS} >= 0, <= demand[p];

# auxiliary variables
var revenue = sum{p in PRODUCTS} cp[p] * y[p];
var cost = sum{r in RESOURCES} cr[r] * x[r];

# objective
maximize profit: revenue - cost;

# constraints
s.t. materials_used {r in RESOURCES}:
    sum{p in PRODUCTS} a[r, p] * y[p] <= x[r];

Overwriting production_planning.mod


In [78]:
import pandas as pd

class ProductionModel(AMPL):
    """
    A class representing a production model using AMPL.
    """
    
    def __init__(self, products, resources, processes):
        """
        Initialize ProductionModel as an AMPL instance.
        
        :param products: A dictionary containing product information.
        :param resources: A dictionary containing resource information.
        :param processes: A dictionary containing process information.
        """
        super(ProductionModel, self).__init__()
        
        # save data in the model instance
        self.products = products
        self.resources = resources
        self.processes = processes
        
        # flag to monitor solution status
        self.solved = False
        
    def load_data(self):
        """
        Prepare the data and pass the information to AMPL.
        """
        # convert the data dictionaries into pandas data frames
        products = pd.DataFrame(self.products).T
        products.rename(columns={'price':'cp'}, inplace=True)
        products.fillna(INF, inplace=True)
        products.index.rename("PRODUCTS", inplace=True)
        
        resources = pd.DataFrame(self.resources).T
        resources.rename(columns={'price':'cr'}, inplace=True)
        resources.fillna(INF, inplace=True)
        resources.index.rename("RESOURCES", inplace=True)
        
        processes = pd.DataFrame(self.processes).T

        # display the generated data frames
        display(products)
        display(resources)
        display(processes)

        # pass data to AMPL
        self.set_data(products, "PRODUCTS")        
        self.set_data(resources, "RESOURCES")        
        self.get_parameter("a").set_values(processes.unstack())
                
    def solve(self, solver=SOLVER):
        """
        Read the model, load the data, set the solver and solve the optimization problem.
        """
        self.read("production_planning.mod")
        self.load_data()
        self.option["solver"] = solver
        super(ProductionModel, self).solve()
        self.solved = True
        
    def report(self):
        """
        Solve, if necessary, then report the model solution.
        """
        if not self.solved:
            self.solve()
            
        print(f"Profit = {self.obj['profit'].value()}")

        print("\nProduction Report")
        y = self.var["y"].get_values().to_pandas()
        y.rename(columns={y.columns[0]: "produced"}, inplace=True)
        y.index.rename("PRODUCTS", inplace=True)
        display(y)

        print("\nResource Report")
        x = self.var["x"].get_values().to_pandas()
        x.rename(columns={x.columns[0]: "consumed"}, inplace=True)
        x.index.rename("RESOURCES", inplace=True)
        display(x)


m = ProductionModel(products, resources, processes)
m.report()


,cp,demand
PRODUCTS,,
U,270.0,40.0
V,210.0,inf


,cr,available
RESOURCES,,
M,10.0,inf
labor A,50.0,100.0
labor B,40.0,80.0


,M,labor A,labor B
U,10,2,1
V,9,1,1


HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 2400
3 simplex iterations
0 barrier iterations
Profit = 2400.0

Production Report


,produced
PRODUCTS,
U,20.0
V,60.0



Resource Report


,consumed
RESOURCES,
M,740.0
labor A,100.0
labor B,80.0
